## Step 1. get out the original CV coordinates from the `cluster_logs.txt` file for each confout 
This may seem a bit backwards, as we first made the histograms, but we essentially need to back-calculate from the cluster logs where on `cv_coordinates` this was, so we can then find the weight per snapshot!!!! 

## Step 2. Convert weights and extract out the relevant cv coordinate indices from these weights
`weights` was originally shaped like `len(cv_coordinates)*2` 

Weights has to do with each **frame** and **not each CV**!!!

Example: `weights[0]`, `weights[2]`... `weights[32]` will all have the same weight. That's because for bead 1, swarm 0-31, they all have the same CV start position. Remember that we did not take the starting bead for any swarm, because we were trying to avoid this sampling bias. 

So, that means in effect that we should skip every other weight for our indexing, as we never take values from the start frame of the swarms


## Step 3. Save weights to each grid histogram

For this, I have decided to then write a textfile into each histogram directory, saving the weights information there. I just feel like this will be easier in the long run for organizational purposes

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import pandas as pd
import MDAnalysis as mda

Obviously, this would be much faster to do together, but I want to keep everything apart for clarity!!!

In [20]:
name_sim = "influx_apo_gate_CV"
simulation_directory = f"/data2/GLUT5_string/string/string_sims/TMD_initial_path/{name_sim}"
cv_coordinates = np.load(f'{simulation_directory}/cv_coordinates.final.MSM.npy')
weights = np.load(f'{simulation_directory}/weights_MSM.npy')
F = np.load(f'{simulation_directory}/F_MSM.npy')
weights_reshaped = weights[1::2]

## I leave this here for you so you can double check yourself one day if you want to
#weights_reshaped = np.vstack((weights[::2], weights[1::2])).T

In [21]:
def process_grid_txt(grid, start = 100):
    infile = f'../confout_files/FES_grids_confouts/{name_sim}/histogram_{grid}/cluster_logs.txt'
    if os.path.exists(infile):
        print(f'Reading grid {grid} confout locations')
        grid = np.loadtxt(infile, dtype=str, ndmin=1)
        processed_confout = []
        for confout in grid:
            confout = confout.split('.')[0]
            confout = confout.split('-')
            iteration = int(confout[0]) - start
            bead = int(confout[1])
            swarm = int(confout[2][1:])
            processed_confout.append([iteration,bead,swarm])
    else:
        print(f'Grid {grid} has no confouts... skipping')
        return

    return(processed_confout)

In [22]:
def get_original_CV(processed_confout):
    
    n_beads = 14
    n_swarms = 32
    
    cv_proj_converted = []
    for grid in processed_confout:   
        iteration_start_cv_proj = grid[0] * (n_beads*n_swarms)
        ## need to remove one from the grid because the actual bead counting will begin 
        ## at 0 on cv_proj, but in the confout files it would start at 1
        ## (just trust me, I checked this very carefully :) )
        bead_start_cv_proj = iteration_start_cv_proj + ((grid[1]-1) * n_swarms)
        swarm_start_cv_proj = bead_start_cv_proj + grid[2]
        cv_proj_converted.append(swarm_start_cv_proj)
        
        #print(f'[{grid[0]+100}, {grid[1]}, {grid[2]}], {swarm_start_cv_proj}')
    return cv_proj_converted
        

In [23]:
grids = np.arange(0,2916)
for grid in grids:
    processed_confout = process_grid_txt(grid = grid)
    if not processed_confout:
        pass
    else:
        cv_coord_from_grid = get_original_CV(processed_confout)
        cv_coord_from_grid_weights = weights_reshaped[cv_coord_from_grid]
        if len(cv_coord_from_grid_weights) != len(processed_confout):
            print("HELLO!!")
        np.savetxt(f'../confout_files/FES_grids_confouts/{name_sim}/histogram_{grid}/weights_per_cv_coord.txt', \
                  cv_coord_from_grid_weights)

Grid 0 has no confouts... skipping
Grid 1 has no confouts... skipping
Grid 2 has no confouts... skipping
Grid 3 has no confouts... skipping
Grid 4 has no confouts... skipping
Grid 5 has no confouts... skipping
Grid 6 has no confouts... skipping
Grid 7 has no confouts... skipping
Grid 8 has no confouts... skipping
Grid 9 has no confouts... skipping
Grid 10 has no confouts... skipping
Grid 11 has no confouts... skipping
Grid 12 has no confouts... skipping
Grid 13 has no confouts... skipping
Grid 14 has no confouts... skipping
Grid 15 has no confouts... skipping
Grid 16 has no confouts... skipping
Grid 17 has no confouts... skipping
Grid 18 has no confouts... skipping
Grid 19 has no confouts... skipping
Grid 20 has no confouts... skipping
Grid 21 has no confouts... skipping
Grid 22 has no confouts... skipping
Grid 23 has no confouts... skipping
Grid 24 has no confouts... skipping
Grid 25 has no confouts... skipping
Grid 26 has no confouts... skipping
Grid 27 has no confouts... skipping
Gr

Reading grid 249 confout locations
Reading grid 250 confout locations
Reading grid 251 confout locations
Reading grid 252 confout locations
Reading grid 253 confout locations
Reading grid 254 confout locations
Reading grid 255 confout locations
Reading grid 256 confout locations
Reading grid 257 confout locations
Reading grid 258 confout locations
Reading grid 259 confout locations
Reading grid 260 confout locations
Reading grid 261 confout locations
Reading grid 262 confout locations
Reading grid 263 confout locations
Reading grid 264 confout locations
Grid 265 has no confouts... skipping
Grid 266 has no confouts... skipping
Grid 267 has no confouts... skipping
Grid 268 has no confouts... skipping
Grid 269 has no confouts... skipping
Grid 270 has no confouts... skipping
Grid 271 has no confouts... skipping
Grid 272 has no confouts... skipping
Grid 273 has no confouts... skipping
Grid 274 has no confouts... skipping
Grid 275 has no confouts... skipping
Grid 276 has no confouts... skipp

Reading grid 496 confout locations
Grid 497 has no confouts... skipping
Grid 498 has no confouts... skipping
Grid 499 has no confouts... skipping
Reading grid 500 confout locations
Reading grid 501 confout locations
Reading grid 502 confout locations
Reading grid 503 confout locations
Reading grid 504 confout locations
Reading grid 505 confout locations
Reading grid 506 confout locations
Reading grid 507 confout locations
Reading grid 508 confout locations
Reading grid 509 confout locations
Reading grid 510 confout locations
Reading grid 511 confout locations
Reading grid 512 confout locations
Reading grid 513 confout locations
Reading grid 514 confout locations
Reading grid 515 confout locations
Reading grid 516 confout locations
Reading grid 517 confout locations
Reading grid 518 confout locations
Reading grid 519 confout locations
Reading grid 520 confout locations
Reading grid 521 confout locations
Reading grid 522 confout locations
Reading grid 523 confout locations
Reading grid 5

Reading grid 728 confout locations
Reading grid 729 confout locations
Reading grid 730 confout locations
Reading grid 731 confout locations
Reading grid 732 confout locations
Reading grid 733 confout locations
Reading grid 734 confout locations
Reading grid 735 confout locations
Reading grid 736 confout locations
Reading grid 737 confout locations
Reading grid 738 confout locations
Reading grid 739 confout locations
Reading grid 740 confout locations
Reading grid 741 confout locations
Reading grid 742 confout locations
Reading grid 743 confout locations
Grid 744 has no confouts... skipping
Grid 745 has no confouts... skipping
Grid 746 has no confouts... skipping
Grid 747 has no confouts... skipping
Grid 748 has no confouts... skipping
Grid 749 has no confouts... skipping
Grid 750 has no confouts... skipping
Grid 751 has no confouts... skipping
Grid 752 has no confouts... skipping
Grid 753 has no confouts... skipping
Grid 754 has no confouts... skipping
Grid 755 has no confouts... skipp

Reading grid 988 confout locations
Reading grid 989 confout locations
Reading grid 990 confout locations
Reading grid 991 confout locations
Reading grid 992 confout locations
Reading grid 993 confout locations
Reading grid 994 confout locations
Reading grid 995 confout locations
Reading grid 996 confout locations
Reading grid 997 confout locations
Reading grid 998 confout locations
Reading grid 999 confout locations
Reading grid 1000 confout locations
Reading grid 1001 confout locations
Reading grid 1002 confout locations
Reading grid 1003 confout locations
Reading grid 1004 confout locations
Grid 1005 has no confouts... skipping
Grid 1006 has no confouts... skipping
Grid 1007 has no confouts... skipping
Grid 1008 has no confouts... skipping
Grid 1009 has no confouts... skipping
Grid 1010 has no confouts... skipping
Grid 1011 has no confouts... skipping
Grid 1012 has no confouts... skipping
Grid 1013 has no confouts... skipping
Grid 1014 has no confouts... skipping
Grid 1015 has no con

...

Reading grid 1516 confout locations
Reading grid 1517 confout locations
Reading grid 1518 confout locations
Reading grid 1519 confout locations
Reading grid 1520 confout locations
Reading grid 1521 confout locations
Reading grid 1522 confout locations
Reading grid 1523 confout locations
Reading grid 1524 confout locations
Reading grid 1525 confout locations
Reading grid 1526 confout locations
Reading grid 1527 confout locations
Reading grid 1528 confout locations
Reading grid 1529 confout locations
Reading grid 1530 confout locations
Grid 1531 has no confouts... skipping
Grid 1532 has no confouts... skipping
Grid 1533 has no confouts... skipping
Grid 1534 has no confouts... skipping
Grid 1535 has no confouts... skipping
Grid 1536 has no confouts... skipping
Grid 1537 has no confouts... skipping
Grid 1538 has no confouts... skipping
Grid 1539 has no confouts... skipping
Grid 1540 has no confouts... skipping
Grid 1541 has no confouts... skipping
Grid 1542 has no confouts... skipping
Grid

Reading grid 1740 confout locations
Reading grid 1741 confout locations
Reading grid 1742 confout locations
Reading grid 1743 confout locations
Reading grid 1744 confout locations
Reading grid 1745 confout locations
Reading grid 1746 confout locations
Grid 1747 has no confouts... skipping
Grid 1748 has no confouts... skipping
Grid 1749 has no confouts... skipping
Grid 1750 has no confouts... skipping
Grid 1751 has no confouts... skipping
Grid 1752 has no confouts... skipping
Grid 1753 has no confouts... skipping
Grid 1754 has no confouts... skipping
Grid 1755 has no confouts... skipping
Grid 1756 has no confouts... skipping
Grid 1757 has no confouts... skipping
Grid 1758 has no confouts... skipping
Grid 1759 has no confouts... skipping
Grid 1760 has no confouts... skipping
Grid 1761 has no confouts... skipping
Grid 1762 has no confouts... skipping
Grid 1763 has no confouts... skipping
Grid 1764 has no confouts... skipping
Grid 1765 has no confouts... skipping
Grid 1766 has no confouts.

Reading grid 2012 confout locations
Reading grid 2013 confout locations
Grid 2014 has no confouts... skipping
Grid 2015 has no confouts... skipping
Grid 2016 has no confouts... skipping
Grid 2017 has no confouts... skipping
Grid 2018 has no confouts... skipping
Grid 2019 has no confouts... skipping
Grid 2020 has no confouts... skipping
Grid 2021 has no confouts... skipping
Grid 2022 has no confouts... skipping
Grid 2023 has no confouts... skipping
Grid 2024 has no confouts... skipping
Grid 2025 has no confouts... skipping
Grid 2026 has no confouts... skipping
Grid 2027 has no confouts... skipping
Grid 2028 has no confouts... skipping
Grid 2029 has no confouts... skipping
Grid 2030 has no confouts... skipping
Grid 2031 has no confouts... skipping
Grid 2032 has no confouts... skipping
Grid 2033 has no confouts... skipping
Grid 2034 has no confouts... skipping
Grid 2035 has no confouts... skipping
Grid 2036 has no confouts... skipping
Grid 2037 has no confouts... skipping
Grid 2038 has no

Reading grid 2276 confout locations
Reading grid 2277 confout locations
Reading grid 2278 confout locations
Reading grid 2279 confout locations
Reading grid 2280 confout locations
Reading grid 2281 confout locations
Grid 2282 has no confouts... skipping
Grid 2283 has no confouts... skipping
Grid 2284 has no confouts... skipping
Grid 2285 has no confouts... skipping
Grid 2286 has no confouts... skipping
Grid 2287 has no confouts... skipping
Grid 2288 has no confouts... skipping
Grid 2289 has no confouts... skipping
Grid 2290 has no confouts... skipping
Grid 2291 has no confouts... skipping
Grid 2292 has no confouts... skipping
Grid 2293 has no confouts... skipping
Grid 2294 has no confouts... skipping
Grid 2295 has no confouts... skipping
Grid 2296 has no confouts... skipping
Grid 2297 has no confouts... skipping
Grid 2298 has no confouts... skipping
Grid 2299 has no confouts... skipping
Grid 2300 has no confouts... skipping
Grid 2301 has no confouts... skipping
Grid 2302 has no confout

Grid 2540 has no confouts... skipping
Grid 2541 has no confouts... skipping
Grid 2542 has no confouts... skipping
Grid 2543 has no confouts... skipping
Grid 2544 has no confouts... skipping
Grid 2545 has no confouts... skipping
Grid 2546 has no confouts... skipping
Grid 2547 has no confouts... skipping
Grid 2548 has no confouts... skipping
Grid 2549 has no confouts... skipping
Grid 2550 has no confouts... skipping
Grid 2551 has no confouts... skipping
Grid 2552 has no confouts... skipping
Grid 2553 has no confouts... skipping
Grid 2554 has no confouts... skipping
Grid 2555 has no confouts... skipping
Grid 2556 has no confouts... skipping
Grid 2557 has no confouts... skipping
Grid 2558 has no confouts... skipping
Grid 2559 has no confouts... skipping
Grid 2560 has no confouts... skipping
Grid 2561 has no confouts... skipping
Grid 2562 has no confouts... skipping
Grid 2563 has no confouts... skipping
Grid 2564 has no confouts... skipping
Grid 2565 has no confouts... skipping
Grid 2566 ha

In [15]:
#process_grid_txt(grid = 49)
np.loadtxt('../confout_files/FES_grids_confouts/influx_BFRU_gate_CV/histogram_49/cluster_logs.txt', dtype = str, ndmin=1)

array(['330-14-s31.clu0'], dtype='<U15')

Now I am testing this out to see how this would work on a practical example, I copied here the code from `TM7b_TM10`

Firstly, the trajectory will be in order that we find in `cv_coord_from_grid`

In [12]:
def theta_of_angle(u, s1, s2, s3):
    from numpy.linalg import norm
    A = u.select_atoms(s1).center_of_geometry()
    B = u.select_atoms(s2).center_of_geometry()
    C = u.select_atoms(s3).center_of_geometry()

    BA = A - B
    BC = C - B
    theta = np.arccos(np.dot(BA, BC)/(norm(BA)*norm(BC)))
    return np.rad2deg(theta)

In [13]:
backbone = ' and name CA' 
condition = 'BFRU'


if os.path.isfile(f'../confout_files/FES_grids_confouts/influx_{condition}_gate_CV/histogram_{grid}/FES_grid_all.xtc'):
    print(f'calculating angles for grid {grid}')
    u = mda.Universe(f'../confout_files/tpr_files/influx_{condition}_gate_CV.wholesys.tpr', \
                    f'../confout_files/FES_grids_confouts/influx_{condition}_gate_CV/histogram_{grid}/FES_grid_all.xtc')

    sels = ['284', '291', '299']
    ## make backbone '' or ' and name CA' to toggle between selections
    theta_u = []
    for ts in u.trajectory:
        theta = theta_of_angle(u, s1 = f'resid {sels[0]}{backbone}', 
                               s2 = f'resid {sels[1]}{backbone}', 
                               s3 = f'resid {sels[2]}{backbone}')

        theta_u.append(theta)

calculating angles for grid 823


In [27]:
theta_u = np.array(theta_u)

In [42]:
theta_times_weights = theta_u * weights_reshaped[cv_coord_from_grid]

In [51]:
sum_theta = np.sum(theta_times_weights)

sum_weights = np.sum(weights_reshaped[cv_coord_from_grid])

In [46]:
sum_weights

0.0004583681930241175

In [53]:
sum_theta / sum_weights

96.35779171471937

In [50]:
np.mean(theta_u)

96.38204165681867